In [1]:
pip install pydantic

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [4]:
#Importing all neccessary libraries 
pip install pydantic
from enum import Enum
from pydantic import BaseModel, Field, field_validator, model_validator,computed_field

#checking if gender is either male or Female
class SexEnum(str, Enum):
    MALE = "Male"
    FEMALE = "Female"

#Main class for evaluating Demographic data
class Demographics(BaseModel):
    #checking if subject id is greater than 0
    subject_id: int = Field(..., gt=0, description="Unique subject ID (must have at least 8 digits)")
    #checking of age is greater than 0 but less than equal to 120 
    age: int = Field(..., ge=0, le=120, description="Age in years")
    #checking if sex is male or female only
    sex: SexEnum = Field(..., description="Sex: Male or Female only")
    #checking of racial group is min length is 2 and max length is 50
    race: str = Field(..., min_length=2, max_length=50, description="Racial group")
    #checking if ethicity is more than 2 but less than 50 , if value not present then american is default value
    ethnicity: str = Field(default="American", min_length=2, max_length=50, description="Ethnic category")
    #checking if weight falls between 1 and 500
    weight_kg: float = Field(..., gt=0, lt=500, description="Weight in kilograms")
    #checking if height falls between 1 and 300
    height_cm: float = Field(..., gt=0, lt=300, description="Height in centimeters")
    #checking if bmi falls between 6 and 59 
    bmi: float = Field(..., gt=5, lt=60, description="Body Mass Index (kg/m²)")
    
    #validating if subject id is greater than len 8 else we are raising an error
    @field_validator('subject_id')
    @classmethod
    def validate_subject_id_length(cls, v):
        if len(str(v)) < 8:
            raise ValueError("subject_id must be at least 8 digits long")
        return v

    #checking if gender is string type and making it case insensitive
    @field_validator('sex', mode='before')
    @classmethod
    def validate_sex_case_insensitive(cls, v):
        if isinstance(v, str):
            v = v.strip().capitalize()
        try:
            return SexEnum(v)
        except ValueError:
            raise ValueError("Sex must be 'Male' or 'Female'")

   
    #returning the category of BMI and adding to final output    
    @computed_field
    @property
    def bmi_category(self) -> str:
        if self.bmi < 18.5:
            return "Underweight"
        elif 18.5 <= self.bmi < 25:
            return "Normal"
        else:
            return "Overweight"
            
    #returning the expected BMI Based on height and weight          
    @computed_field
    @property
    def actual_bmi(self) -> float:
        height_m = self.height_cm / 100
        return round(self.weight_kg / (height_m ** 2), 1)



In [5]:
data = {
    "subject_id": 10011004,
    "age": 33,
    "sex": "FEMALE",
    "race": "Japanese",
    # Ethnicity not provided → defaults to "American"
    "weight_kg":56 ,
    "height_cm": 156,
    "bmi":21.0
}

demo = Demographics(**data)
print(demo.model_dump())  # expected_bmi will be included!



{'subject_id': 10011004, 'age': 33, 'sex': <SexEnum.FEMALE: 'Female'>, 'race': 'Japanese', 'ethnicity': 'American', 'weight_kg': 56.0, 'height_cm': 156.0, 'bmi': 21.0, 'bmi_category': 'Normal', 'actual_bmi': 23.0}


In [8]:
data = {
    "subject_id": 10011004,
    "age": 33,
    "sex": "FEMALE",
    "race": "Japanese",
    "Ethnicity": "",
    "weight_kg":56 ,
    "height_cm": 156,
    "bmi":21.0
}

demo = Demographics(**data)
print(demo.model_dump())  # expected_bmi will be included!

{'subject_id': 10011004, 'age': 33, 'sex': <SexEnum.FEMALE: 'Female'>, 'race': 'Japanese', 'ethnicity': 'American', 'weight_kg': 56.0, 'height_cm': 156.0, 'bmi': 21.0, 'bmi_category': 'Normal', 'actual_bmi': 23.0}


In [9]:
data = {
    "subject_id": 1,
    "age": 33,
    "sex": "FEMALE",
    "race": "Japanese",
    "Ethnicity": "",
    "weight_kg":56 ,
    "height_cm": 156,
    "bmi":21.0
}

demo = Demographics(**data)
print(demo.model_dump())  # expected_bmi will be included!

ValidationError: 1 validation error for Demographics
subject_id
  Value error, subject_id must be at least 8 digits long [type=value_error, input_value=1, input_type=int]
    For further information visit https://errors.pydantic.dev/2.8/v/value_error

In [10]:
data = {
    "subject_id": 10011004,
    "age": 33,
    "sex": "Others",
    "race": "Japanese",
    "Ethnicity": "",
    "weight_kg":56 ,
    "height_cm": 156,
    "bmi":21.0
}

demo = Demographics(**data)
print(demo.model_dump())

ValidationError: 1 validation error for Demographics
sex
  Value error, Sex must be 'Male' or 'Female' [type=value_error, input_value='Others', input_type=str]
    For further information visit https://errors.pydantic.dev/2.8/v/value_error